# Segmenting and Clustering Neighborhoods in Toronto

Question 1

In [32]:
import pandas as pd
import numpy as np

In [33]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

Deleting the rows with "Not assigned" in the Borough column

In [34]:
df=df.replace('Not assigned',np.NaN)
df=df.dropna(subset=["Borough"], axis=0) 
df.reset_index(drop=True, inplace=True)  

Grouping all the neighborhoods with the same postal code

In [35]:
df1=df.groupby('Postal Code')['Neighbourhood'].apply(list).reset_index(name='Neighbourhood')

Merging both dataframes only for the unique postal codes

In [36]:
df2 = pd.merge(df1, df, how='outer', on=['Postal Code'])
df2=df2[['Postal Code','Neighbourhood_x','Borough']]
df2.rename(columns={'Neighbourhood_x':'Neighbourhood'},inplace=True)

In [37]:
postals=df2[['Postal Code']]
neighs=df2[['Neighbourhood']]
boroughs=df2[['Borough']]

Assigning the borough name to the neighborhoods labeled "Not assigned"

In [38]:
for neigh, borough in zip(neighs , boroughs):
    if neigh == "Not Assigned":
        neigh = borough

In [39]:
df2

,Postal Code,Neighbourhood,Borough
0,M1B,"[Malvern, Rouge]",Scarborough
1,M1C,"[Rouge Hill, Port Union, Highland Creek]",Scarborough
2,M1E,"[Guildwood, Morningside, West Hill]",Scarborough
3,M1G,[Woburn],Scarborough
4,M1H,[Cedarbrae],Scarborough
5,M1J,[Scarborough Village],Scarborough
6,M1K,"[Kennedy Park, Ionview, East Birchmount Park]",Scarborough
7,M1L,"[Golden Mile, Clairlea, Oakridge]",Scarborough
8,M1M,"[Cliffside, Cliffcrest, Scarborough Village West]",Scarborough
9,M1N,"[Birch Cliff, Cliffside West]",Scarborough


In [40]:
df2.shape

(103, 3)

Question 2

In [41]:
df3 = pd.read_csv("http://cocl.us/Geospatial_data")

Merging the dataframe containing the latitude and longitude of postal codes with the dataframe of the unique postal codes

In [42]:
df4 = pd.merge(df2, df3, how='outer', on=['Postal Code'])

In [43]:
df4

,Postal Code,Neighbourhood,Borough,Latitude,Longitude
0,M1B,"[Malvern, Rouge]",Scarborough,43.806686,-79.194353
1,M1C,"[Rouge Hill, Port Union, Highland Creek]",Scarborough,43.784535,-79.160497
2,M1E,"[Guildwood, Morningside, West Hill]",Scarborough,43.763573,-79.188711
3,M1G,[Woburn],Scarborough,43.770992,-79.216917
4,M1H,[Cedarbrae],Scarborough,43.773136,-79.239476
5,M1J,[Scarborough Village],Scarborough,43.744734,-79.239476
6,M1K,"[Kennedy Park, Ionview, East Birchmount Park]",Scarborough,43.727929,-79.262029
7,M1L,"[Golden Mile, Clairlea, Oakridge]",Scarborough,43.711112,-79.284577
8,M1M,"[Cliffside, Cliffcrest, Scarborough Village West]",Scarborough,43.716316,-79.239476
9,M1N,"[Birch Cliff, Cliffside West]",Scarborough,43.692657,-79.264848


Question 3

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

In [ ]:
CLIENT_ID = ' YPVTKOPCYL5VI1NOKP5MZZ2NMCWATJK2502GY1MPWCB0PD4I ' 
CLIENT_SECRET = ' VOVPY3V1ZA5B11PMQLKFYAY0ZNROQTY4TCJ1U1GR1LAXZMFO '
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def getNearbyVenues(neighs, latitudes, longitudes, radius=500):  
    venues_list=[]
    for neigh, lat, lng in zip(neighs, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
    # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=df4['Neighbourhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters